In [68]:

import os
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
input_rtf= os.path.join(os.getcwd(), "data", "Tabla 1.6 Esclerosis multiple.rtf")
import pandas as pd
from striprtf.striprtf import rtf_to_text
from scipy.spatial.distance import mahalanobis
from scipy.stats import chi2

with open(input_rtf, 'r', encoding='utf-8') as file:
    rtf_content = file.read()
text_content = rtf_to_text(rtf_content)
lines = text_content.strip().split('\n')
data = [line.split(' ') for line in lines]
data = pd.DataFrame(data[1:])  # Primera fila como encabezado
data=data[[0,2,4,6,8,10]]
data.columns = ['edad','t1ao','d1ao','t2ao','d2ao','grupo']
data['edad']=data['edad'].astype('float')
data['t1ao']=data['t1ao'].astype('float')
data['d1ao']=data['d1ao'].astype('float')
data['t2ao']=data['t2ao'].astype('float')
data['d2ao']=data['d2ao'].astype('float')
data['grupo']=data['grupo'].astype('float')

In [69]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   edad    97 non-null     float64
 1   t1ao    97 non-null     float64
 2   d1ao    97 non-null     float64
 3   t2ao    97 non-null     float64
 4   d2ao    97 non-null     float64
 5   grupo   97 non-null     float64
dtypes: float64(6)
memory usage: 4.7 KB


In [70]:
data.describe()

,edad,t1ao,d1ao,t2ao,d2ao,grupo
count,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000
mean,39.412371,156.503093,4.764948,207.929897,5.063918,0.298969
std,15.192345,23.017803,10.868456,28.943451,11.477716,0.460184
min,19.000000,125.400000,0.000000,169.200000,0.000000,0.000000
25%,26.000000,141.400000,0.800000,188.000000,0.400000,0.000000
50%,36.000000,148.800000,1.600000,201.200000,1.600000,0.000000
75%,50.000000,163.200000,3.400000,217.600000,3.400000,1.000000
max,79.000000,238.400000,90.200000,328.000000,83.000000,1.000000


In [61]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
data['muestra'] = data.index
def plot_scatter_subplots(data, configs, cols=2):
    rows = (len(configs) + cols - 1) // cols  
    fig = make_subplots(
        rows=rows, cols=cols, 
        subplot_titles=[cfg['title'] for cfg in configs]
    )
    
   
    for i, cfg in enumerate(configs):
        row = i // cols + 1  # Fila actual
        col = i % cols + 1   # Columna actual
        scatter = go.Scatter(x=data[cfg['x']], y=data[cfg['y']], mode='markers', name=cfg['title'])
        fig.add_trace(scatter, row=row, col=col)
    
    # Ajustar el diseño
    fig.update_layout(
        height=400 * rows,  # Ajustar altura de la figura
        title_text="Scatterplots ",
        showlegend=False
    )
    fig.show()
# Configuración de los subplots
scatter_configs = [
    {'x': 'muestra', 'y': 'edad', 'title': 'muestra vs edad'},
    {'x': 'muestra', 'y': 't1ao', 'title': 'muestra vs t1ao'},
    {'x': 'muestra', 'y': 'd1ao', 'title': 'muestra vs d1ao'},
    {'x': 'muestra', 'y': 't2ao', 'title': 'muestra vs t2ao'},
    {'x': 'muestra', 'y': 'd2ao', 'title': 'muestra vs d2ao'},
    {'x': 'muestra', 'y': 'grupo', 'title': 'muestra vs grupo'}
]

# Llamar a la función
plot_scatter_subplots(data, scatter_configs)

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
data['muestra'] = data.index
def plot_histogram_subplots(data, configs, cols=2):
    rows = (len(configs) + cols - 1) // cols  # Redondear hacia arriba
    fig = make_subplots(
        rows=rows, cols=cols, 
        subplot_titles=[cfg['title'] for cfg in configs]
    )
    for i, cfg in enumerate(configs):
        row = i // cols + 1  # Fila actual
        col = i % cols + 1   # Columna actual
        histogram = go.Histogram(x=data[cfg['x']], name=cfg['title'])
        fig.add_trace(histogram, row=row, col=col)
    fig.update_layout(
        height=400 * rows,  # Ajustar altura de la figura
        title_text="Histogramas",
        showlegend=False,
        template='plotly_white'
    )
    fig.show()

histogram_configs = [
    {'x': 'edad', 'title': 'edad'},
    {'x': 't1ao', 'title': 't1ao'},
    {'x': 'd1ao', 'title': 'd1ao'},
    {'x': 't2ao', 'title': 't2ao'},
    {'x': 'd2ao', 'title': 'd2ao'},
    {'x': 'grupo', 'title': 'grupo'}
]

# Llamar a la función
plot_histogram_subplots(data, histogram_configs)

In [85]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def univariate_outlier_detection(data, var):
    # Calcular los cuartiles y el IQR
    Q1 = np.percentile(data[var], 25)
    Q3 = np.percentile(data[var], 75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Identificar outliers
    data['Outlier'] = (data[var] < lower_bound) | (data[var] > upper_bound)
    data_no_outliers = data[~data['Outlier']]
    mean_no_outliers = data_no_outliers[var].mean()

    # Guardar los datos originales para el primer gráfico
    data_original = data.copy()

    # Reemplazar outliers con la media de los datos no atípicos para el segundo gráfico
    data['Replaced'] = np.where(data['Outlier'], True, False)  # Marcar los reemplazados
    data[var] = np.where(data['Outlier'], mean_no_outliers, data[var])

    # Crear los subplots
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=[f'Original con Outliers ({var})', f'Corregido ({var})']
    )

    # Gráfico 1: Original con outliers coloreados
    scatter_original_inliers = go.Scatter(
        x=data_original[~data_original['Outlier']].index,
        y=data_original[~data_original['Outlier']][var],
        mode='markers',
        marker=dict(color='blue', size=8),
        name='Inliers (Original)'
    )
    scatter_original_outliers = go.Scatter(
        x=data_original[data_original['Outlier']].index,
        y=data_original[data_original['Outlier']][var],
        mode='markers',
        marker=dict(color='red', size=8),
        name='Outliers (Original)'
    )
    fig.add_trace(scatter_original_inliers, row=1, col=1)
    fig.add_trace(scatter_original_outliers, row=1, col=1)

    # Límites en el gráfico original
    fig.add_shape(
        type='line', x0=data_original.index.min(), x1=data_original.index.max(),
        y0=lower_bound, y1=lower_bound,
        line=dict(color='blue', dash='dot', width=2),
        row=1, col=1
    )
    fig.add_shape(
        type='line', x0=data_original.index.min(), x1=data_original.index.max(),
        y0=upper_bound, y1=upper_bound,
        line=dict(color='green', dash='dot', width=2),
        row=1, col=1
    )

    # Gráfico 2: Corregido (con puntos reemplazados)
    scatter_corrected_inliers = go.Scatter(
        x=data[~data['Replaced']].index,
        y=data[~data['Replaced']][var],
        mode='markers',
        marker=dict(color='blue', size=8),
        name='Inliers (Corregido)'
    )
    scatter_corrected_outliers = go.Scatter(
        x=data[data['Replaced']].index,
        y=data[data['Replaced']][var],
        mode='markers',
        marker=dict(color='red', size=8),
        name='Outliers Reemplazados'
    )
    fig.add_trace(scatter_corrected_inliers, row=1, col=2)
    fig.add_trace(scatter_corrected_outliers, row=1, col=2)

    # Límites y media en el gráfico corregido
    fig.add_shape(
        type='line', x0=data.index.min(), x1=data.index.max(),
        y0=lower_bound, y1=lower_bound,
        line=dict(color='blue', dash='dot', width=2),
        row=1, col=2
    )
    fig.add_shape(
        type='line', x0=data.index.min(), x1=data.index.max(),
        y0=upper_bound, y1=upper_bound,
        line=dict(color='green', dash='dot', width=2),
        row=1, col=2
    )
    fig.add_shape(
        type='line', x0=data.index.min(), x1=data.index.max(),
        y0=mean_no_outliers, y1=mean_no_outliers,
        line=dict(color='red', dash='dash', width=2),
        row=1, col=2
    )

    # Configurar diseño del gráfico
    fig.update_layout(
        title_text=f"Análisis de Outliers para {var}",
        height=500, width=1200,
        showlegend=True
    )
    fig.show()

    # Eliminar las columnas auxiliares
    data = data.drop(columns=['Outlier', 'Replaced'])

    return data

In [87]:
for i in ['edad','t1ao',	'd1ao'	,'t2ao',	'd2ao',	'grupo']:
    univariate_outlier_detection(data,i)

In [100]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import mahalanobis
from scipy.stats import chi2
import plotly.express as px

def mahalanobis_outlier_detection(data, x_var, y_var, p_value=0.95):
    selected_data = data[[x_var, y_var]].dropna()  # Filtrar datos nulos
    mean = selected_data.mean().values
    cov_matrix = np.cov(selected_data.values, rowvar=False)
    inv_cov_matrix = np.linalg.inv(cov_matrix)
    selected_data['Mahalanobis'] = selected_data.apply(
        lambda row: mahalanobis(row.values, mean, inv_cov_matrix), axis=1
    )
    chi2_critical = chi2.ppf(p_value, df=2)  # Grados de libertad = número de variables (2)
    selected_data['Outlier'] = selected_data['Mahalanobis'] > np.sqrt(chi2_critical)
    fig = px.scatter(
        selected_data,
        x=x_var,
        y=y_var,
        color='Outlier',
        title='Distancia de Mahalanobis y Valores Atípicos',
        labels={'Outlier': 'Es Atípico'},
        template='plotly_white'
    )
    fig.show()
    return selected_data
result = mahalanobis_outlier_detection(data, x_var='t1ao', y_var='t2ao', p_value=0.95)